In [78]:
# -*- coding: utf-8 -*-
import pymysql
import pandas as pd
from pymysql.cursors import DictCursor

class config:
    user = 'root'
    host = '127.0.0.1'
    passwd = '!019Takuma'
    db = 'HRdata'

    
#mysqlのデータベースと接続
dbcon = pymysql.connect(
    database = config.db,
    user = config.user,
    password = config.passwd,
    host = config.host,
    charset ="utf8"
)
dbcur = dbcon.cursor(DictCursor)

In [61]:
#dataの獲得

dbcur.execute("select result_race.race_id,finishing_time from result_race")
df = pd.DataFrame(dbcur.fetchall())
df.head()

,finishing_time,race_id
0,89.6,1
1,89.8,1
2,89.9,1
3,90.1,1
4,90.4,1


### 先に小規模データで実験する。

In [ ]:
db_test = pd.DataFrame({'id':[1,1,1,2,2,3], 'score':[12,0,8,10,5,11]})
db_test

In [ ]:
db_test_formin = db_test[db_test['score'] != 0]
db_test_formin

In [ ]:
data_min = {'id':[x for x in range(1, len(db_test_formin.groupby('id'))+ 1)] ,
            'score_min':[x for x in db_test_formin.groupby('id')['score'].min()]}
print(data_min)

In [ ]:
db_min = pd.DataFrame(data_min)
db_min

In [ ]:
db_test = pd.merge(db_test , db_min)
db_test.head()

In [ ]:
db_test['score_to_top'] = db_test['score'] - db_test['score_min']
db_test.head()

### では本番。

In [62]:
db_for_min = df[df['finishing_time'] > 0]
db_for_min.head()

,finishing_time,race_id
0,89.6,1
1,89.8,1
2,89.9,1
3,90.1,1
4,90.4,1


In [63]:
data_min = {'race_id':[x for x in range(1, len(db_for_min.groupby('race_id'))+ 1)] ,
            'top_time':[x for x in db_for_min.groupby('race_id')['finishing_time'].min()]}
df_min = pd.DataFrame(data_min)
df_min.head()

,race_id,top_time
0,1,89.6
1,2,59.1
2,3,104.8
3,4,71.2
4,5,105.3


In [64]:
df = pd.merge(df,df_min)
df.head()

,finishing_time,race_id,top_time
0,89.6,1,89.6
1,89.8,1,89.6
2,89.9,1,89.6
3,90.1,1,89.6
4,90.4,1,89.6


In [65]:
df['time_to_top'] = df['finishing_time'] - df['top_time']
df.head()

,finishing_time,race_id,top_time,time_to_top
0,89.6,1,89.6,0.0
1,89.8,1,89.6,0.2
2,89.9,1,89.6,0.3
3,90.1,1,89.6,0.5
4,90.4,1,89.6,0.8


In [72]:
dbcur.execute("select result_race.horse_id,first_order,corner_order,last_speed from result_race")
df_rest = pd.DataFrame(dbcur.fetchall())
df_rest.head()

,corner_order,first_order,horse_id,last_speed
0,1,1,2005100161,36.1
1,3,5,2005101066,35.9
2,3,2,2005109144,36.1
3,2,2,2005102851,36.5
4,5,6,2005100911,36.3


In [73]:
df_all = pd.concat([df, df_rest], axis=1)
df_all.head()

,finishing_time,race_id,top_time,time_to_top,corner_order,first_order,horse_id,last_speed
0,89.6,1,89.6,0.0,1,1,2005100161,36.1
1,89.8,1,89.6,0.2,3,5,2005101066,35.9
2,89.9,1,89.6,0.3,3,2,2005109144,36.1
3,90.1,1,89.6,0.5,2,2,2005102851,36.5
4,90.4,1,89.6,0.8,5,6,2005100911,36.3


In [74]:
df_all = df_all.ix[:, ['race_id','horse_id','first_order','corner_order','finishing_time','last_speed','time_to_top']]
df_all.head()

,race_id,horse_id,first_order,corner_order,finishing_time,last_speed,time_to_top
0,1,2005100161,1,1,89.6,36.1,0.0
1,1,2005101066,5,3,89.8,35.9,0.2
2,1,2005109144,2,3,89.9,36.1,0.3
3,1,2005102851,2,2,90.1,36.5,0.5
4,1,2005100911,6,5,90.4,36.3,0.8


In [76]:
#一旦クリア
dbcur.execute('delete from result_race')

#Databaseへ格納
L = df_all.values.tolist()
for row in L:
    dbcur.execute('INSERT INTO result_race(race_id,horse_id,first_order,corner_order,finishing_time,last_speed,time_to_top) VALUES (%s,%s,%s,%s,%s,%s,%s)',row)

In [77]:
#保存と終了処理
dbcon.commit()
dbcon.close()